# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [1]:
client_key = 'WafAnFjAtTg51Vb8I94pWFRb6'
client_secret = 'T8VibzLMgMuMDvNkScLBSUSpSV6NhwlJh1GrLxy8jzaer8sUb7'

In [2]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAI2%2BewEAAAAAcjftd9OTNEE68Y8DBga5KsKHzzs%3DXcM6HqhrHIm47ay7usG4DXgNU3TsT4tRYUOakT2mZDgAyUWAoL'}

In [6]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
max_tweets=100
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [8]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [9]:
climate_tweets = response.json()


In [10]:
len(climate_tweets['data'])

100

In [11]:
tweets +=climate_tweets['data']


In [12]:

 ids = [tw['id'] for tw in climate_tweets['data']]
    

In [13]:
min_id = min(ids)

In [14]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [24]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                'until_id': min_id
              }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['statuses']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

In [25]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 100


# Let's print some tweets

In [26]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [27]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '1545046309551235073',
  'conversation_id': '1548763199838846980',
  'created_at': '2022-07-17T20:20:47.000Z',
  'entities': {'mentions': [{'end': 16,
                             'id': '31047836',
                             'start': 0,
                             'username': 'michaeljknowles'}]},
  'id': '1548764675122216961',
  'in_reply_to_user_id': '31047836',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 0},
  'referenced_tweets': [{'id': '1548763199838846980', 'type': 'replied_to'}],
  'reply_settings': 'everyone',
  'source': 'Twitter for iPhone',
  'text': '@michaeljknowles Let’s help the move the climate change goal post. '
          'Say - all the way off the cliff?'},
 {'author_id': '2757343023',
  'conversation_id': '1548764675034185728',
  'created_at': '2022-07-17T20:20:47.000Z',
  'entities': {'mentions': [{'end': 14,
             

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [28]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

@michaeljknowles Let’s help the move the climate change goal post. Say - all the way off the cliff?

RT @Botanygeek: Honestly, wouldn’t it be better if politicians in this race acted like climate change was the greatest threat to face human…

RT @PGDynes: A chart for the “1976 was just as hot” headbangers. Stop burying your head in the hot sand. The climate is experiencing rapid…

RT @Botanygeek: Honestly, wouldn’t it be better if politicians in this race acted like climate change was the greatest threat to face human…



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [29]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['conversation_id', 'referenced_tweets', 'id', 'reply_settings', 'entities', 'public_metrics', 'source', 'created_at', 'lang', 'author_id', 'text'])
11


In [31]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [38]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [39]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'conversation_id': '1548763199838846980',
  'referenced_tweets': [{'type': 'replied_to', 'id': '1548763199838846980'}],
  'in_reply_to_user_id': '31047836',
  'id': '1548764675122216961',
  'reply_settings': 'everyone',
  'entities': {'mentions': [{'start': 0,
     'end': 16,
     'username': 'michaeljknowles',
     'id': '31047836'}]},
  'public_metrics': {'retweet_count': 0,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'source': 'Twitter for iPhone',
  'created_at': '2022-07-17T20:20:47.000Z',
  'lang': 'en',
  'text': '@michaeljknowles Let’s help the move the climate change goal post. Say - all the way off the cliff?',
  'user_id': '1545046309551235073',
  'referenced_tweets_ids': '1548763199838846980',
  'retweet_count': 0},
 {'conversation_id': '1548764675034185728',
  'referenced_tweets': [{'type': 'retweeted', 'id': '1548571208219652096'}],
  'id': '1548764675034185728',
  'reply_settings': 'everyone',
  'entities': {'mentions': [{'start': 3,
     'end': 14

In [40]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [41]:
climate_feed_df

,conversation_id,referenced_tweets,in_reply_to_user_id,id,reply_settings,entities,public_metrics,source,created_at,lang,text,user_id,referenced_tweets_ids,retweet_count,geo
0,1548763199838846980,"[{'type': 'replied_to', 'id': '154876319983884...",31047836,1548764675122216961,everyone,"{'mentions': [{'start': 0, 'end': 16, 'usernam...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,2022-07-17T20:20:47.000Z,en,@michaeljknowles Let’s help the move the clima...,1545046309551235073,1548763199838846980,0,NaN
1,1548764675034185728,"[{'type': 'retweeted', 'id': '1548571208219652...",NaN,1548764675034185728,everyone,"{'mentions': [{'start': 3, 'end': 14, 'usernam...","{'retweet_count': 1941, 'reply_count': 0, 'lik...",Twitter for Android,2022-07-17T20:20:47.000Z,en,"RT @Botanygeek: Honestly, wouldn’t it be bette...",2757343023,1548571208219652096,1941,NaN
2,1548764674941927426,"[{'type': 'retweeted', 'id': '1548692433155809...",NaN,1548764674941927426,everyone,"{'mentions': [{'start': 3, 'end': 11, 'usernam...","{'retweet_count': 46, 'reply_count': 0, 'like_...",Twitter for iPhone,2022-07-17T20:20:47.000Z,en,RT @PGDynes: A chart for the “1976 was just as...,2813006618,1548692433155809281,46,NaN
3,1548764673029259265,"[{'type': 'retweeted', 'id': '1548571208219652...",NaN,1548764673029259265,everyone,"{'mentions': [{'start': 3, 'end': 14, 'usernam...","{'retweet_count': 1941, 'reply_count': 0, 'lik...",Twitter for Android,2022-07-17T20:20:47.000Z,en,"RT @Botanygeek: Honestly, wouldn’t it be bette...",745379722587217920,1548571208219652096,1941,NaN
4,1548764666746257409,"[{'type': 'retweeted', 'id': '1548315624098566...",NaN,1548764666746257409,everyone,"{'mentions': [{'start': 3, 'end': 14, 'usernam...","{'retweet_count': 15774, 'reply_count': 0, 'li...",Twitter for Android,2022-07-17T20:20:45.000Z,en,RT @WhipRealer: It’s a farce. Always has been....,1375129529098862593,1548315624098566144,15774,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,1548764470578667526,"[{'type': 'retweeted', 'id': '1548638641408610...",NaN,1548764470578667526,everyone,"{'mentions': [{'start': 3, 'end': 14, 'usernam...","{'retweet_count': 1758, 'reply_count': 0, 'lik...",Twitter for iPhone,2022-07-17T20:19:59.000Z,en,"RT @JamesGDyke: Bear in mind the heatwaves, wi...",13725202,1548638641408610304,1758,NaN
91,1548764469928550402,"[{'type': 'retweeted', 'id': '1548707824645115...",NaN,1548764469928550402,everyone,"{'mentions': [{'start': 3, 'end': 19, 'usernam...","{'retweet_count': 97, 'reply_count': 0, 'like_...",Twitter for iPhone,2022-07-17T20:19:58.000Z,en,RT @DeTocqueville14: It's kinda suspicious tha...,712449097458823168,1548707824645115905,97,NaN
92,1548764467693080576,"[{'type': 'retweeted', 'id': '1548435892859047...",NaN,1548764467693080576,everyone,"{'mentions': [{'start': 3, 'end': 17, 'usernam...","{'retweet_count': 25, 'reply_count': 0, 'like_...",Twitter for Android,2022-07-17T20:19:58.000Z,fr,RT @JerryJarrell9: @RepAdamSchiff Manmade clim...,28276119,1548435892859047936,25,NaN
93,1548764462722719745,"[{'type': 'retweeted', 'id': '1548415680935968...",NaN,1548764462722719745,everyone,"{'mentions': [{'start': 3, 'end': 15, 'usernam...","{'retweet_count': 86, 'reply_count': 0, 'like_...",Twitter Web App,2022-07-17T20:19:57.000Z,en,RT @Yodaonexile: @WhipRealer If the climate ch...,1542597765422895104,1548415680935968768,86,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [43]:
# write code here datatype listing
climate_feed_df.dtypes

conversation_id          object
referenced_tweets        object
in_reply_to_user_id      object
id                       object
reply_settings           object
entities                 object
public_metrics           object
source                   object
created_at               object
lang                     object
text                     object
user_id                  object
referenced_tweets_ids    object
retweet_count             int64
geo                      object
dtype: object

# Write columns with wrong datatype and mention correct data type too.

All id's should be either string or integer. Tweets/texts should be strings. 


# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [126]:
def tweet_user_generator(tweet_user_list):
    for user in tweet_user_list:
        try:
            yield user
        except KeyError:
            continue

In [127]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

In [128]:
user_df

,0
0,data
1,includes
2,meta


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [142]:
# write code here for datatype listing
user_df.dtypes

0    object
dtype: object

# write columns with wrong datatype and mention correct data type too.

0 should be string

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [132]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]

In [133]:
retweeted_status_df.head()

,id,retweet_count
0,1548764675122216961,0
1,1548764675034185728,1941
2,1548764674941927426,46
3,1548764673029259265,1941
4,1548764666746257409,15774


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [141]:
# Write code here
(retweeted_status_df['retweet_count'] == 0).sum()

17